# Balance Risk Set Matching
Detablan, Paul France M.

# Balanced Risk Set Matching (Li et al., 2001)
This research examines the use of cystoscopy and hydrodistention as treatments for interstitial cystitis, a chronic, non-life-threatening condition characterized by persistent symptoms. The study aims to compare treatment outcomes among patients with similar symptom histories but differing timelines of receiving interventions. The journal article emphasizes analyzing how the timing of treatment administration influences patient results, exploring whether earlier or later therapeutic approaches yield distinct effects on symptom management and recovery.

Link to article: [Balanced Risk Set Matching: Journal of the American Statistical Association](https://doi.org/10.1198/016214501753208573)

# Programming Assignment (Data Analytics)

This assignment is a test of your algorithmic thinking. You can use any A.I. tools ot assist you in this coding assignment.

Instructions:

- This is done by pair, preferably your thesis partner.
- Each person should create a Github Repo titled 'Assignment_1_Data_Analytics'.
- Read the journal provided.
- Develop a Python implementation of the procedures in the journal.
- Deadline is before premidterm week.

## install packages

In [ ]:
pip install 

## Import required packages

In [2]:
# Import Required Libraries

# Import numerical and data analysis libraries
import numpy as np
import pandas as pd

# Include distance computation and utilities
from scipy.spatial.distance import cdist
from collections import defaultdict

# Add plotting capabilities
import matplotlib.pyplot as plt

## Initialization of Global Variables and Seed

In [9]:
# Initialize Global Variables & Settings

# Set the random seed for reproducibility
np.random.seed(30)

# Define the number of patients and evaluation periods
number_of_patients = 210
evaluation_years = 4
evaluation_months = evaluation_years * 12
MAX_MATCHES = 100

# Titles for data visualization
BOX_TITLES = [
    'Baseline',
    'At Treatment',
    '3 Months after Treatment',
    '6 Months after Treatment',
    'Difference (3 mos posttreatment)',
    'Difference (6 mos posttreatment)'
]

# Labels for categorizing groups
BOX_LABELS = ['Never/Later Treated', 'Treated']

## Initialize patient assessment data at the time of entry

In [10]:
# Generate Sample Patient Data

# Create synthetic baseline data for patients
baseline_data = pd.DataFrame({
    "patient_id": np.arange(0, number_of_patients),
    "gender": np.random.choice(['M', 'F'], number_of_patients),
    "pain": np.random.randint(0, 10, number_of_patients),
    "urgency": np.random.randint(0, 10, number_of_patients),
    "frequency": np.random.randint(0, 20, number_of_patients)
})

# Display the first few rows of the baseline data
baseline_data.head()

,patient_id,gender,pain,urgency,frequency
0,0,F,1,7,1
1,1,F,3,1,4
2,2,F,3,2,16
3,3,F,3,7,3
4,4,M,6,6,11


Patients undergo evaluations approximately every 3 months for a period of up to 4 years. During each assessment, three key metrics are recorded:

- Pain Level
- Urgency Score
- Nocturnal Frequency

Both pain level and urgency score are subjective ratings measured on a scale from 0 to 9.

## Conduct evaluations at 3-month intervals

In [11]:
# Create synthetic evaluation data for patients at 3-month intervals
evaluation_data = pd.DataFrame()

for patient_id in range(number_of_patients):
    chosen_treatment_time = np.random.choice(list(np.arange(3, evaluation_months + 1, 3)) + [None])
    
    # Evaluate every 3 months up to 4 years
    for month in range(3, evaluation_months + 1, 3):
        pain = np.random.randint(0, 10, 1)
        urgency = np.random.randint(0, 10, 1)
        frequency = np.random.randint(0, 20, 1)
        time_since_entry = month

        if chosen_treatment_time is None or month < chosen_treatment_time:
            time_treated = None
            treated = 0
        else:
            time_treated = chosen_treatment_time
            treated = 1

        evaluation_data = pd.concat([evaluation_data, pd.DataFrame({
            'patient_id': patient_id, 
            'pain': pain, 
            'urgency': urgency, 
            'frequency': frequency, 
            'time_since_entry': time_since_entry, 
            'time_treated': time_treated, 
            'treated': treated
        })])

# Display the first few rows of the evaluation data
evaluation_data.groupby('patient_id')[['pain', 'urgency', 'frequency']].mean().head()

,pain,urgency,frequency
patient_id,,,
0,4.5625,5.1250,7.6875
1,4.5000,4.8125,8.3125
2,4.8125,5.6875,9.4375
3,5.4375,3.4375,10.3750
4,4.6875,4.0000,9.2500


## Initialize risk sets

In [12]:
# Create Risk Sets

# Initialize an empty dictionary to store risk sets
risk_sets = {}

# Loop through unique treatment times, excluding missing values
for treatment_time in evaluation_data['time_treated'].dropna().unique():
    # Identify patients who started treatment at the current time point
    treated_patients = evaluation_data[
        (evaluation_data['time_since_entry'] == treatment_time) & 
        (evaluation_data['time_treated'] == treatment_time)
    ]
    
    # Identify patients who remained untreated at the same time point
    untreated_patients = evaluation_data[
        (evaluation_data['time_since_entry'] == treatment_time) & 
        (evaluation_data['treated'] == 0)
    ]

    # Store the treated and untreated groups in the dictionary
    risk_sets[treatment_time] = (treated_patients, untreated_patients)

# Display the available time points in the risk sets
risk_sets.keys()

dict_keys([27, 18, 33, 21, 48, 3, 39, 42, 30, 36, 45, 6, 12, 24, 9, 15])

## Binary variable function 

In [21]:
# Binary Variable Functions

def create_binary_variables(df, variables):
    binary_vars = defaultdict(list)
    for var in variables:
        # Compute one-third and two-thirds threshold
        lower_threshold = np.percentile(df[var], 33)
        upper_threshold = np.percentile(df[var], 67)

        # Create binary variables for lower, middle, and upper groups
        binary_lower = (df[var] <= lower_threshold).astype(int)
        binary_middle = ((df[var] > lower_threshold) & (df[var] <= upper_threshold)).astype(int)
        binary_upper = (df[var] > upper_threshold).astype(int)
        
        # Append to binary_vars "x_lower" is already implied if both variables are 0
        binary_vars[f"{var}_middle"].extend(binary_middle)
        binary_vars[f"{var}_upper"].extend(binary_upper)

    # Generate binary DataFrame for each symptom variable
    binary_df = pd.DataFrame(binary_vars)
    return binary_df

## Risk set updated

In [14]:
# Update Risk Sets with Binary Variables

# Define the variables to be used for creating binary indicators
variables = [
    'pain_current',
    'urgency_current',
    'frequency_current',
    'pain_baseline',
    'urgency_baseline',
    'frequency_baseline'
]

# Initialize a new dictionary to store updated risk sets
updated_risk_sets = {}

# Loop through each risk set and update with binary variables
for key, (treated, untreated) in risk_sets.items():
    # Merge treated patients with baseline data
    treated_patients = treated.merge(baseline_data, on='patient_id', suffixes=['_current', '_baseline'])
    
    # Generate binary variables for treated patients
    binary_treated = create_binary_variables(treated_patients, variables)
    treated_patients = pd.concat([treated_patients, binary_treated], axis=1)
    
    # Merge untreated patients with baseline data
    untreated_patients = untreated.merge(baseline_data, on='patient_id', suffixes=['_current', '_baseline'])
    
    # Generate binary variables for untreated patients
    binary_untreated = create_binary_variables(untreated_patients, variables)
    untreated_patients = pd.concat([untreated_patients, binary_untreated], axis=1)
    
    # Store the updated treated and untreated groups in the new dictionary
    updated_risk_sets[key] = (treated_patients, untreated_patients)

# Display the columns of the first updated treated group to verify the changes
list(updated_risk_sets.values())[0][0].columns

Index(['patient_id', 'pain_current', 'urgency_current', 'frequency_current',
       'time_since_entry', 'time_treated', 'treated', 'gender',
       'pain_baseline', 'urgency_baseline', 'frequency_baseline',
       'pain_current_middle', 'pain_current_upper', 'urgency_current_middle',
       'urgency_current_upper', 'frequency_current_middle',
       'frequency_current_upper', 'pain_baseline_middle',
       'pain_baseline_upper', 'urgency_baseline_middle',
       'urgency_baseline_upper', 'frequency_baseline_middle',
       'frequency_baseline_upper'],
      dtype='object')

## Matching Functions

Implementing matching algorithm using minimum cost flow and balanced pair approaches

### Computing for the different distance between the treated and untreated using Mahalanobis

In [16]:
# Compute Mahalanobis Distance
def compute_mahalanobis_distance(treated, untreated, covariates):
    treated_matrix = treated[covariates].to_numpy()
    untreated_matrix = untreated[covariates].to_numpy()
    
    return cdist(treated_matrix, untreated_matrix, metric='mahalanobis')

# Store distances and matches as well as average distance
distance_dict = defaultdict(list)

for key, (treated, untreated) in updated_risk_sets.items():
    distance_matrix = compute_mahalanobis_distance(treated, untreated, variables)
    
    for i, row in enumerate(distance_matrix):
        for j, distance in enumerate(row):
            distance_dict[distance].append((int(treated['patient_id'].iloc[i]), int(untreated['patient_id'].iloc[j])))

average_distance = sum(distance_dict.keys()) / len(distance_dict.keys())


In [17]:
# Create matches from Minimum Cost Flow in a Network (top 100)
mcf_matches = []  # Minimum Cost Flow matches
count = 0

for keys in sorted(distance_dict.keys()):
    for match in distance_dict[keys]:
        if count >= MAX_MATCHES:
            break
            
        mcf_matches.append(match)
        count += 1

# Compute sum of K binary variables
def find_sum_bin(series, variables):
    total = 0

    # This way lower = 0, middle = 1, upper = 2
    for i, var in enumerate(variables):
        total += (i + 1) * series.loc[var]

    return total

In [18]:
# Balanced Pair Matching
bpm_matches = []
patient_set = set()
count = 0

binary_vars = [
    f"{x}_middle" for x in variables
] + [
    f"{x}_upper" for x in variables
]

for (treated, untreated) in updated_risk_sets.values():
    for i in range(treated.shape[0]):
        for j in range(untreated.shape[0]):
            # If balanced
            if count < MAX_MATCHES and find_sum_bin(treated.iloc[i], binary_vars) == find_sum_bin(untreated.iloc[j], binary_vars):
                treated_id = int(treated['patient_id'].iloc[i])
                untreated_id = int(untreated['patient_id'].iloc[j])

                if treated_id not in patient_set and untreated_id not in patient_set: 
                    bpm_matches.append((treated_id, untreated_id))
                    
                    patient_set.add(treated_id)
                    patient_set.add(untreated_id)
                    count += 1


## Grouping of Data Frames(basing from the matches)

In [19]:
# Group DataFrames based on matches
def find_dataframes(match):
    x, y = match
    features = ['pain', 'urgency', 'frequency']

    # Getting baseline data
    baseline_match_x = baseline_data[baseline_data['patient_id'] == x][features]
    baseline_match_y = baseline_data[baseline_data['patient_id'] == y][features]

    features.append('treated')
    
    # Getting on treatment data
    treatment_match_x = evaluation_data[(evaluation_data['patient_id'] == x) 
        & (evaluation_data['time_treated'] == evaluation_data['time_since_entry'])][features + ['time_since_entry']]

    time_since_entry = treatment_match_x['time_since_entry'].iloc[0] if not treatment_match_x['time_since_entry'].empty else -1
    
    if time_since_entry == -1:
        return ()
    else:
        treatment_match_x = treatment_match_x.drop('time_since_entry', axis=1)

    treatment_match_y = evaluation_data[(evaluation_data['patient_id'] == y) 
        & (evaluation_data['time_since_entry'] == time_since_entry)][features]

    # Getting 3 months after data
    time_since_entry += 3
    
    _3_months_match_x = evaluation_data[(evaluation_data['patient_id'] == x) 
        & (evaluation_data['time_since_entry'] == time_since_entry)][features]
    _3_months_match_y = evaluation_data[(evaluation_data['patient_id'] == y) 
        & (evaluation_data['time_since_entry'] == time_since_entry)][features]

    # Getting 6 months after data
    time_since_entry += 3

    _6_months_match_x = evaluation_data[(evaluation_data['patient_id'] == x) 
        & (evaluation_data['time_since_entry'] == time_since_entry)][features]
    _6_months_match_y = evaluation_data[(evaluation_data['patient_id'] == y) 
        & (evaluation_data['time_since_entry'] == time_since_entry)][features]

    return (
        baseline_match_x, 
        baseline_match_y,
        treatment_match_x,
        treatment_match_y,
        _3_months_match_x,
        _3_months_match_y,
        _6_months_match_x,
        _6_months_match_y
    )

In [20]:
# Initializing DataFrames
dataframes = [pd.DataFrame() for _ in range(12)]

treatment_index = 2
month_index = 4

for match in bpm_matches:
    data = find_dataframes(match)

    if len(data) == 0:
        continue

    for i in range(len(dataframes) - 4):  # minus 4 to remove last 2 box plots
        dataframes[i] = pd.concat([dataframes[i], data[i]])

    for i in range(4):
        index = len(dataframes) - 4 + i
        dataframes[index] = pd.concat([dataframes[index], data[month_index + i] - data[treatment_index + (i // 2)]])

for index in range(len(dataframes)):
    dataframes[index] = dataframes[index].dropna()